# 3. OpenAI Image Target

`OpenAIImageTarget` supports two different modes:
- Generating a brand new image from a text prompt.
- Editing an existing image (or combining a set of images) from a text prompt.

## Generating Images (Text --> Image)

This example shows how to use the image target to create an image from a text prompt.

In [ ]:
from pyrit.executor.attack import (
    AttackExecutor,
    AttackScoringConfig,
    ConsoleAttackResultPrinter,
    PromptSendingAttack,
)
from pyrit.prompt_target import OpenAIChatTarget, OpenAIImageTarget
from pyrit.score import SelfAskTrueFalseScorer, TrueFalseQuestion
from pyrit.setup import IN_MEMORY, initialize_pyrit_async

await initialize_pyrit_async(memory_db_type=IN_MEMORY)  # type: ignore

img_prompt_target = OpenAIImageTarget()

image_scorer = SelfAskTrueFalseScorer(
    chat_target=OpenAIChatTarget(),
    true_false_question=TrueFalseQuestion(true_description="The response matches the objective {{objective}}"),
)

scoring_config = AttackScoringConfig(objective_scorer=image_scorer)

attack = PromptSendingAttack(
    objective_target=img_prompt_target,
    attack_scoring_config=scoring_config,
)

objectives = [
    "Give me an image of a raccoon pirate as a Spanish baker in Spain",
    "Give me an image of a cafe in Paris",
]

results = await AttackExecutor().execute_attack_async(  # type: ignore
    attack=attack,
    objectives=objectives,
)
for result in results:
    await ConsoleAttackResultPrinter().print_summary_async(result=result)  # type: ignore
    print(result.last_response.converted_value)

Found default environment files: ['C:\\Users\\frdubut\\.pyrit\\.env', 'C:\\Users\\frdubut\\.pyrit\\.env.local']
Loaded environment file: C:\Users\frdubut\.pyrit\.env
Loaded environment file: C:\Users\frdubut\.pyrit\.env.local

 Attack Summary 
────────────────────────────────────────────────────────────────────────────────────────────────────
  📋 Basic Information
    • Objective: Give me an image of a raccoon pirate as a Spanish baker in Spain
    • Attack Type: PromptSendingAttack
    • Conversation ID: 1dbad4cf-4408-4675-9257-933445f1eec3

  ⚡ Execution Metrics
    • Turns Executed: 1
    • Execution Time: 53.61s

  🎯 Outcome
    • Status: ✅ SUCCESS
    • Reason: Objective achieved according to scorer

   Final Score
    Scorer: SelfAskTrueFalseScorer
    • Category: N/A
    • Type: true_false
    • Value: true
    • Rationale:
      The image fulfills the objective as it shows a raccoon depicted as a pirate baker in a
      Spanish setting, complete with appropriate attire and surr

## Editing Images (Text + Image --> Image)

This example shows how to use the image target to edit an existing image (or combine a set of images) from a text prompt.

In [ ]:
from pyrit.models import SeedGroup, SeedPrompt

# use the previously generated images as seeds
image_seeds = [
    SeedPrompt(
        value=result.last_response.converted_value,
        data_type="image_path",
    )
    for result in results
]

# this is a single request with three parts, one text and two images
all_seeds = [
    SeedPrompt(
        value="Make the character in the first image fit in the cafe in the second image",
        data_type="text",
    )
] + image_seeds

seed_group = SeedGroup(seeds=all_seeds)

result = await attack.execute_async(
    objective=seed_group.prompts[0].value,
    next_message=seed_group.next_message,
)  # type: ignore
await ConsoleAttackResultPrinter().print_summary_async(result=result)  # type: ignore
print(result.last_response.converted_value)


 Attack Summary 
────────────────────────────────────────────────────────────────────────────────────────────────────
  📋 Basic Information
    • Objective: Make the character in the first image fit in the cafe in the second image
    • Attack Type: PromptSendingAttack
    • Conversation ID: ae3ef4e7-ab56-4e79-a38a-a5ca224ad1af

  ⚡ Execution Metrics
    • Turns Executed: 1
    • Execution Time: 1m 16s

  🎯 Outcome
    • Status: ✅ SUCCESS
    • Reason: Objective achieved according to scorer

   Final Score
    Scorer: SelfAskTrueFalseScorer
    • Category: N/A
    • Type: true_false
    • Value: true
    • Rationale:
      The pirate raccoon character's attire, expressions, and activity—drinking a beverage,
      sitting in a Parisian cafe setting, alongside croissants and wine—blend well
      aesthetically with the decor and ambiance of the French cafe. The lighting, detailing,
      and perspective align harmoniously, making the character feel natural within the scene.
C:\Source\Py